In [3]:
import csv
import cv2
import numpy as np

In [9]:
"""Extract data from the .csv file"""
lines = []
with open('./data/driving_log.csv') as file:
    reader = csv.reader(file)
    for line in reader:
        lines.append(line)
# images and mearsurments will be our input_X and input_y
# Augment training set by flip the image horizonaly and flip the sign of mearsurment.
augmented_images = []
augmented_mearsurements = []
for i in range(len(lines)):
    if i == 0:
        continue
    line = lines[i]
    for j in range(3):
        # source path could be center(0)/left(1)/right(2)
        source_path = line[j]
        filename = source_path.split('/')[-1]
        current_path = './data/IMG/' + filename
        image = cv2.imread(current_path)
        augmented_images.append(image)
        augmented_images.append(cv2.flip(image, 1))
        if j == 0:
            mearsurement = float(line[3])
        elif j == 1:
            mearsurement = float(line[3]) + 0.2
        elif j == 2:
            mearsurement = float(line[3]) - 0.2
        augmented_mearsurements.append(mearsurement)
        augmented_mearsurements.append(mearsurement * -1.0)


# for i in range(len(lines)):
#     if i == 0:
#         continue
#     line = lines[i]
#     source_path = line[0]
#     filename = source_path.split('/')[-1]
#     current_path = './data/IMG/' + filename
#     image = cv2.imread(current_path)
#     augmented_images.append(image)
#     augmented_images.append(cv2.flip(image, 1))
#     mearsurement = float(line[3])
#     augmented_mearsurements.append(mearsurement)
#     augmented_mearsurements.append(mearsurement * -1.0)

In [10]:
"""Set X_train and y_train"""
X_train = np.array(augmented_images)
y_train = np.array(augmented_mearsurements)
print(X_train.shape, y_train.shape)

(48216, 160, 320, 3) (48216,)


In [11]:
print(y_train[0:12])

[ 0.  -0.   0.2 -0.2 -0.2  0.2  0.  -0.   0.2 -0.2 -0.2  0.2]


In [15]:
"""Build a simple regression model"""
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Convolution2D, MaxPooling2D, Cropping2D
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard  


model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
# crop 70 rows of pixels on the top and 25 rows of pixels on the bottom.
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Convolution2D(24, 5, 5, subsample=(2,2),activation='relu'))     # subsample: stride
# model.add(MaxPooling2D())
model.add(Convolution2D(36, 5, 5, subsample=(2,2),activation='relu'))
# model.add(MaxPooling2D())
model.add(Convolution2D(48, 5, 5, subsample=(2,2),activation='relu'))
# model.add(MaxPooling2D())
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
# output regression layer
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')


/home/nqiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  if sys.path[0] == '':
/home/nqiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  
/home/nqiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  app.launch_new_instance()
/home/nqiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/home/nqiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


In [17]:
callbacks = [EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')]

model.fit(X_train, y_train, 
          validation_split=0.2, 
          shuffle=True, 
          batch_size=128,
          epochs=10,
          callbacks=callbacks)



model.save("./models/model_aug_3_2e.h5")



Train on 38572 samples, validate on 9644 samples
Epoch 1/10
38572/38572 [==============================] - 23s 601us/step - loss: 0.0114 - val_loss: 0.0190
Epoch 2/10
38572/38572 [==============================] - 23s 603us/step - loss: 0.0107 - val_loss: 0.0185
Epoch 3/10
38572/38572 [==============================] - 23s 606us/step - loss: 0.0099 - val_loss: 0.0188


In [7]:
K.clear_session()